# (a) First, consider the TechFit Smartwatch with the price response functions provided in Table 2 for the first two weeks (assume no cross-elasticity). Using the KKT conditions, derive the optimal prices, assuming they are non-negative but otherwise have no restrictions.

In [1]:
import pandas as pd
from gurobipy import Model, GRB, LinExpr

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract data for Week 1 and Week 2 for TechFit Smartwatch
week1_intercept = 1000
week1_coeff = -5
week2_intercept = 950
week2_coeff = -4.5

# Create the optimization model
model = Model("Optimal_Pricing")

# Add decision variables for prices in Week 1 and Week 2
P1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Week1")
P2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Week2")

# Define the revenue functions for both weeks
revenue_week1 = P1 * (week1_intercept + week1_coeff * P1)
revenue_week2 = P2 * (week2_intercept + week2_coeff * P2)

# Set the objective to maximize total revenue
model.setObjective(revenue_week1 + revenue_week2, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Price for Week 1: ${P1.X:.2f}")
    print(f"Optimal Price for Week 2: ${P2.X:.2f}")
    print(f"Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Set parameter Username
Set parameter LicenseID to value 2609998
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x01fefae1
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+03, 1e+03]
  QObjective range [9e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective 1.00138889e+05
Optimal Price for Week 1: $100.00
Optimal Price for Week 2: $105.56
Maximum Total Revenue: $100138.89


# (b) Consider again the TechFit Smartwatch with the same price response functions as in the previous question for the first two weeks (assume no cross-elasticity). Using the KKT conditions, derive the optimal prices assuming they are non-negative but must remain the same across both weeks.

In [2]:
import pandas as pd
from gurobipy import Model, GRB

# Load the data from the provided CSV file
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract data for Week 1 and Week 2 for TechFit Smartwatch
week1_intercept = 1000
week1_coeff = -5
week2_intercept = 950
week2_coeff = -4.5

# Create the optimization model
model = Model("Optimal_Same_Price")

# Add a decision variable for the common price in both weeks
P = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Common_Price")

# Define the total revenue function with the same price across both weeks
total_revenue = P * (week1_intercept + week1_coeff * P) + P * (week2_intercept + week2_coeff * P)

# Set the objective to maximize total revenue
model.setObjective(total_revenue, GRB.MAXIMIZE)

# Optimize the model
model.optimize()

# Check and print the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Common Price for Week 1 and Week 2: ${P.X:.2f}")
    print(f"Maximum Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x36313fc2
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+03, 2e+03]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective 1.00065789e+05
Optimal Common Price for Week 1 and Week 2: $102.63
Maximum Total Revenue: $100065.79


# (c) What do you observe about the optimal prices derived using the KKT conditions with and without the equality constraint? Based on our class discussion from the Variable Pricing with Diversion example, why does this occur? What is the benefit of dynamic pricing?

### **(c) Answer – Observations and Analysis**

We can observe the following differences between the results in **(a)** and **(b)**:

---

### **1. Observations on Optimal Prices and Revenue**

1. **Optimal Prices (Without Equality Constraint):**
   - **Week 1 Optimal Price (\( P_1 \)):** $100.00  
   - **Week 2 Optimal Price (\( P_2 \)):** $105.56  
   - **Maximum Total Revenue:** $100,138.89  

2. **Optimal Price (With Equality Constraint):**
   - **Common Price (\( P_1 = P_2 \)):** $102.63  
   - **Maximum Total Revenue:** $100,065.79  

3. **Revenue Difference:**  
   The maximum total revenue without the equality constraint is **$100,138.89**, which is **$73.10 higher** than the revenue with the equality constraint ($100,065.79). This is because dynamic pricing allows each week to have a different optimal price, better aligning with the demand-response functions for those weeks.

---

### **2. Explanation – Why This Occurs**
- **Variable Pricing Advantage:**  
  In (a), we allowed the prices for Week 1 and Week 2 to be optimized independently. This flexibility enables the price to better match the demand curve for each week, resulting in higher revenue.  
  In (b), the same price is enforced across both weeks, which represents a trade-off between the two demand curves, leading to a slightly suboptimal outcome in both weeks.  

- **Demand Curves Matter:**  
  - In Week 1, the optimal price was $100, which aligned perfectly with the demand curve (Intercept: 1000, Own-Price Coefficient: -5).  
  - In Week 2, the demand curve had a different structure (Intercept: 950, Own-Price Coefficient: -4.5), leading to a different optimal price of $105.56 when optimized independently.

- **KKT Conditions and Flexibility:**  
  - Without the equality constraint, the KKT conditions found the true optimal solution for each week.  
  - Adding the constraint \( P_1 = P_2 \) introduces a compromise, where the price ($102.63) falls between the two optimal prices from (a).

---

### **3. Benefit of Dynamic Pricing**
- **Higher Revenue Potential:**  
  As shown by the difference in total revenue, dynamic pricing allows the company to extract more value by adapting prices to weekly demand patterns. This flexibility is critical in maximizing revenue.  
- **Better Demand Matching:**  
  By adjusting prices weekly, the company can better capture the willingness to pay of different consumer segments at different times.  
- **Seasonality and Market Responsiveness:**  
  Dynamic pricing enables the company to respond to seasonality and promotional events (e.g., Black Friday), further enhancing profitability.  

---

### **Conclusion**
- **Static Pricing** simplifies operations but sacrifices potential revenue.  
- **Dynamic Pricing** offers superior performance by allowing prices to vary in response to demand changes, leading to optimal results across different market conditions.

---

If you'd like, I can also help visualize these differences using a plot comparing the price and revenue under the two scenarios. 😊

# (d) Now consider both products. Using the price response functions in the price response.csv file for the first two weeks only, determine the optimal prices using the projected gradient descent algorithm. For each product, assume static pricing across both weeks. Initialize all prices at zero, with a step size of 0.001 and a stopping criterion of 10−6. What are the optimal prices?

In [3]:
# import pandas as pd
# import numpy as np

# # Load the data
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Filter data for the first two weeks and both products
# techfit_weeks = df[(df['Product'] == 'TechFit Smartwatch') & (df['Week'] <= 2)]
# powersound_weeks = df[(df['Product'] == 'PowerSound Earbuds') & (df['Week'] <= 2)]

# # Extract coefficients
# intercept_techfit = techfit_weeks['Intercept'].mean()
# coeff_techfit = techfit_weeks['Own_Price_Coefficient'].mean()

# intercept_powersound = powersound_weeks['Intercept'].mean()
# coeff_powersound = powersound_weeks['Own_Price_Coefficient'].mean()

# # Initialization
# P_techfit = 0.0
# P_powersound = 0.0
# step_size = 0.001
# tolerance = 1e-6
# max_iter = 100000

# # Projected Gradient Descent
# for i in range(max_iter):
#     # Compute gradients
#     grad_techfit = intercept_techfit + 2 * coeff_techfit * P_techfit
#     grad_powersound = intercept_powersound + 2 * coeff_powersound * P_powersound
    
#     # Update prices
#     new_P_techfit = P_techfit + step_size * grad_techfit
#     new_P_powersound = P_powersound + step_size * grad_powersound
    
#     # Projection to ensure non-negative prices
#     new_P_techfit = max(0, new_P_techfit)
#     new_P_powersound = max(0, new_P_powersound)
    
#     # Check stopping criterion
#     if np.linalg.norm([new_P_techfit - P_techfit, new_P_powersound - P_powersound]) < tolerance:
#         break
    
#     # Update current prices
#     P_techfit, P_powersound = new_P_techfit, new_P_powersound

# # Print the optimal prices
# print(f"Optimal Price for TechFit Smartwatch: ${P_techfit:.2f}")
# print(f"Optimal Price for PowerSound Earbuds: ${P_powersound:.2f}")


In [4]:
# from gurobipy import Model, GRB

# # Create the optimization model
# model = Model("Optimal_Pricing_PGD_Verification")

# # Add decision variables for prices
# P_techfit = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_TechFit")
# P_powersound = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_PowerSound")

# # Define the revenue functions for both products
# revenue_techfit = P_techfit * (intercept_techfit + coeff_techfit * P_techfit)
# revenue_powersound = P_powersound * (intercept_powersound + coeff_powersound * P_powersound)

# # Set the objective to maximize total revenue
# model.setObjective(revenue_techfit + revenue_powersound, GRB.MAXIMIZE)

# # Optimize the model
# model.optimize()

# # Print the optimal prices and maximum revenue
# if model.status == GRB.OPTIMAL:
#     print(f"Gurobi - Optimal Price for TechFit Smartwatch: ${P_techfit.X:.2f}")
#     print(f"Gurobi - Optimal Price for PowerSound Earbuds: ${P_powersound.X:.2f}")
#     print(f"Gurobi - Maximum Total Revenue: ${model.objVal:.2f}")
# else:
#     print("No optimal solution found.")


In [5]:
# import pandas as pd
# import numpy as np

# # Load the data from the provided CSV file
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Filter data for Week 1 and Week 2
# data_week1 = df[df['Week'] == 1]
# data_week2 = df[df['Week'] == 2]

# # Initialize prices for both products at zero
# price_smartwatch = 0.0
# price_earbuds = 0.0

# # Set step size and stopping criterion
# step_size = 0.001
# tolerance = 1e-6
# max_iterations = 10000

# def calculate_demand(intercept, own_coeff, cross_coeff, own_price, cross_price):
#     """ Calculate demand based on the given coefficients and prices. """
#     return intercept + own_coeff * own_price + cross_coeff * cross_price

# def calculate_revenue(intercept, own_coeff, cross_coeff, own_price, cross_price):
#     """ Calculate revenue as price times demand. """
#     demand = calculate_demand(intercept, own_coeff, cross_coeff, own_price, cross_price)
#     return own_price * demand

# def gradient_descent(prices, intercepts, own_coeffs, cross_coeffs, step_size, tolerance, max_iterations):
#     """ Perform Projected Gradient Descent to optimize prices. """
#     prev_prices = np.array(prices)
#     for iteration in range(max_iterations):
#         gradients = []
#         for i in range(2):
#             own_price = prev_prices[i]
#             cross_price = prev_prices[1 - i]
#             # Calculate partial derivative of revenue with respect to the own price
#             demand = calculate_demand(intercepts[i], own_coeffs[i], cross_coeffs[i], own_price, cross_price)
#             gradient = demand + own_price * own_coeffs[i]
#             gradients.append(gradient)
        
#         # Update prices using the gradients
#         new_prices = prev_prices + step_size * np.array(gradients)
        
#         # Project prices to be non-negative
#         new_prices = np.maximum(new_prices, 0)
        
#         # Check for convergence
#         if np.linalg.norm(new_prices - prev_prices) < tolerance:
#             print(f"Converged in {iteration} iterations.")
#             break
#         prev_prices = new_prices
    
#     return new_prices

# # Coefficients and intercepts for TechFit Smartwatch and PowerSound Earbuds for Week 1 and Week 2
# intercepts = [data_week1['Intercept'].values[0], data_week1['Intercept'].values[1]]
# own_coeffs = [data_week1['Own_Price_Coefficient'].values[0], data_week1['Own_Price_Coefficient'].values[1]]
# cross_coeffs = [data_week1['Cross_Price_Coefficient'].values[0], data_week1['Cross_Price_Coefficient'].values[1]]

# # Perform gradient descent
# optimal_prices = gradient_descent(
#     prices=[price_smartwatch, price_earbuds],
#     intercepts=intercepts,
#     own_coeffs=own_coeffs,
#     cross_coeffs=cross_coeffs,
#     step_size=step_size,
#     tolerance=tolerance,
#     max_iterations=max_iterations
# )

# print(f"Optimal Price for TechFit Smartwatch: ${optimal_prices[0]:.2f}")
# print(f"Optimal Price for PowerSound Earbuds: ${optimal_prices[1]:.2f}")


Converged in 4605 iterations.
Optimal Price for TechFit Smartwatch: $107.63
Optimal Price for PowerSound Earbuds: $94.93


In [6]:
# import pandas as pd
# from gurobipy import Model, GRB, QuadExpr

# # Load the data from the provided CSV file
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Filter data for Week 1 and Week 2
# data_week1 = df[df['Week'] == 1]
# data_week2 = df[df['Week'] == 2]

# # Extract coefficients and intercepts for both products
# intercept_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Intercept'].values[0]
# own_coeff_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Own_Price_Coefficient'].values[0]
# cross_coeff_smartwatch = data_week1[data_week1['Product'] == 'TechFit Smartwatch']['Cross_Price_Coefficient'].values[0]

# intercept_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Intercept'].values[0]
# own_coeff_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Own_Price_Coefficient'].values[0]
# cross_coeff_earbuds = data_week1[data_week1['Product'] == 'PowerSound Earbuds']['Cross_Price_Coefficient'].values[0]

# # Create the optimization model
# model = Model("Optimal_Pricing_with_Cross_Elasticity")

# # Add decision variables for the prices of both products
# P_smartwatch = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_TechFit_Smartwatch")
# P_earbuds = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_PowerSound_Earbuds")

# # Define the revenue functions for both products
# revenue_smartwatch = P_smartwatch * (intercept_smartwatch + own_coeff_smartwatch * P_smartwatch + cross_coeff_smartwatch * P_earbuds)
# revenue_earbuds = P_earbuds * (intercept_earbuds + own_coeff_earbuds * P_earbuds + cross_coeff_earbuds * P_smartwatch)

# # Set the objective to maximize total revenue (combined for both products)
# model.setObjective(revenue_smartwatch + revenue_earbuds, GRB.MAXIMIZE)

# # Optimize the model
# model.optimize()

# # Check and print the results
# if model.status == GRB.OPTIMAL:
#     print(f"Optimal Price for TechFit Smartwatch: ${P_smartwatch.X:.2f}")
#     print(f"Optimal Price for PowerSound Earbuds: ${P_earbuds.X:.2f}")
#     print(f"Maximum Total Revenue: ${model.objVal:.2f}")
# else:
#     print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x54120bf9
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+02, 3e+02]
  QObjective range [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -5.03006734e+06  6.16825626e+06  8.21e+02 1.84e+01  9.82e+05     0s
   1  -3.

In [10]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Extract data for Week 1 and Week 2 for both products
week_data = df[df['Week'].isin([1, 2])]
intercept_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Intercept'].mean()
own_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Own_Price_Coefficient'].mean()
cross_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Cross_Price_Coefficient'].mean()

intercept_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Intercept'].mean()
own_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Own_Price_Coefficient'].mean()
cross_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Cross_Price_Coefficient'].mean()

# Initialize prices at zero
P_sw = 0.0
P_eb = 0.0
step_size = 0.001
tolerance = 1e-6
max_iter = 10000

# Projected Gradient Descent Algorithm
for i in range(max_iter):
    # Calculate demands
    demand_sw = intercept_sw + own_coeff_sw * P_sw + cross_coeff_sw * P_eb
    demand_eb = intercept_eb + own_coeff_eb * P_eb + cross_coeff_eb * P_sw

    # Calculate gradients (partial derivatives of revenue functions)
    grad_sw = (intercept_sw + 2 * own_coeff_sw * P_sw + cross_coeff_sw * P_eb)
    grad_eb = (intercept_eb + 2 * own_coeff_eb * P_eb + cross_coeff_eb * P_sw)

    # Update prices using the gradient
    new_P_sw = max(0, P_sw + step_size * grad_sw)  # Ensure non-negativity
    new_P_eb = max(0, P_eb + step_size * grad_eb)  # Ensure non-negativity

    # Check convergence
    if abs(new_P_sw - P_sw) < tolerance and abs(new_P_eb - P_eb) < tolerance:
        break

    # Update prices
    P_sw = new_P_sw
    P_eb = new_P_eb

# Print optimal prices
print(f"Optimal Static Price for TechFit Smartwatch: ${P_sw:.2f}")
print(f"Optimal Static Price for PowerSound Earbuds: ${P_eb:.2f}")


Optimal Static Price for TechFit Smartwatch: $89.32
Optimal Static Price for PowerSound Earbuds: $106.95


In [11]:
# import pandas as pd
# from gurobipy import Model, GRB

# # Load the data
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Extract data for Week 1 and Week 2 for both products
# week_data = df[df['Week'].isin([1, 2])]

# # Compute average coefficients for both weeks for each product
# intercept_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Intercept'].mean()
# own_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Own_Price_Coefficient'].mean()
# cross_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Cross_Price_Coefficient'].mean()

# intercept_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Intercept'].mean()
# own_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Own_Price_Coefficient'].mean()
# cross_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Cross_Price_Coefficient'].mean()

# # Create the optimization model
# model = Model("Optimal_Pricing_MultiProduct")

# # Add decision variables for prices
# P_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Smartwatch")
# P_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Earbuds")

# # Define the revenue functions for both products
# revenue_sw = P_sw * (intercept_sw + own_coeff_sw * P_sw + cross_coeff_sw * P_eb)
# revenue_eb = P_eb * (intercept_eb + own_coeff_eb * P_eb + cross_coeff_eb * P_sw)

# # Set the objective to maximize total revenue
# model.setObjective(revenue_sw + revenue_eb, GRB.MAXIMIZE)

# # Optimize the model
# model.optimize()

# # Check and print the results
# if model.status == GRB.OPTIMAL:
#     print(f"Optimal Static Price for TechFit Smartwatch: ${P_sw.X:.2f}")
#     print(f"Optimal Static Price for PowerSound Earbuds: ${P_eb.X:.2f}")
#     print(f"Maximum Total Revenue: ${model.objVal:.2f}")
# else:
#     print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x717b369e
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+02, 3e+02]
  QObjective range [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.10096739e+06  5.18706769e+06  8.61e+02 1.76e+01  9.83e+05     0s
   1  -3.

In [12]:
# import pandas as pd
# import numpy as np
# from gurobipy import Model, GRB

# # Load the data from the provided CSV file
# df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# # Extract data for Week 1 and Week 2 for both products
# week_data = df[df['Week'].isin([1, 2])]

# # Calculate average intercepts and coefficients for both products
# intercept_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Intercept'].mean()
# own_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Own_Price_Coefficient'].mean()
# cross_coeff_sw = week_data[week_data['Product'] == 'TechFit Smartwatch']['Cross_Price_Coefficient'].mean()

# intercept_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Intercept'].mean()
# own_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Own_Price_Coefficient'].mean()
# cross_coeff_eb = week_data[week_data['Product'] == 'PowerSound Earbuds']['Cross_Price_Coefficient'].mean()

# # Projected Gradient Descent (PGD) Implementation
# print("Projected Gradient Descent (PGD) Method:")

# # Initialize prices at zero
# P_sw, P_eb = 0.0, 0.0
# step_size = 0.001
# tolerance = 1e-6
# max_iter = 10000

# # Projected Gradient Descent Algorithm
# for i in range(max_iter):
#     # Calculate demands
#     demand_sw = intercept_sw + own_coeff_sw * P_sw + cross_coeff_sw * P_eb
#     demand_eb = intercept_eb + own_coeff_eb * P_eb + cross_coeff_eb * P_sw

#     # Calculate gradients
#     grad_sw = (intercept_sw + 2 * own_coeff_sw * P_sw + cross_coeff_sw * P_eb)
#     grad_eb = (intercept_eb + 2 * own_coeff_eb * P_eb + cross_coeff_eb * P_sw)

#     # Update prices with gradient step and ensure non-negativity
#     new_P_sw = max(0, P_sw + step_size * grad_sw)
#     new_P_eb = max(0, P_eb + step_size * grad_eb)

#     # Check convergence
#     if abs(new_P_sw - P_sw) < tolerance and abs(new_P_eb - P_eb) < tolerance:
#         break

#     # Update prices
#     P_sw, P_eb = new_P_sw, new_P_eb

# # Print PGD results
# print(f"Optimal Static Price for TechFit Smartwatch (PGD): ${P_sw:.2f}")
# print(f"Optimal Static Price for PowerSound Earbuds (PGD): ${P_eb:.2f}")

# # Gurobi Method
# print("\nGurobi Optimization Method:")

# # Create the optimization model
# model = Model("Optimal_Pricing_MultiProduct")

# # Add decision variables for prices
# P_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Smartwatch")
# P_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Price_Earbuds")

# # Define the revenue functions with cross-price elasticity
# revenue_sw = P_sw * (intercept_sw + own_coeff_sw * P_sw + cross_coeff_sw * P_eb)
# revenue_eb = P_eb * (intercept_eb + own_coeff_eb * P_eb + cross_coeff_eb * P_sw)

# # Set the objective to maximize total revenue
# model.setObjective(revenue_sw + revenue_eb, GRB.MAXIMIZE)

# # Optimize the model
# model.optimize()

# # Check and print the Gurobi results
# if model.status == GRB.OPTIMAL:
#     print(f"Optimal Price for TechFit Smartwatch (Gurobi): ${P_sw.X:.2f}")
#     print(f"Optimal Price for PowerSound Earbuds (Gurobi): ${P_eb.X:.2f}")
#     print(f"Maximum Total Revenue (Gurobi): ${model.objVal:.2f}")
# else:
#     print("No optimal solution found.")


Projected Gradient Descent (PGD) Method:
Optimal Static Price for TechFit Smartwatch (PGD): $89.32
Optimal Static Price for PowerSound Earbuds (PGD): $106.95

Gurobi Optimization Method:
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x717b369e
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+02, 3e+02]
  QObjective range [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Obje

# (e) For the full model, why is this optimization problem considered to be a nonlinear program? Discuss why no linear reformulations of the problem are possible.

### **(e) Why is this optimization problem a Nonlinear Program (NLP)?**

The optimization problem in this assignment is considered a **Nonlinear Program (NLP)** due to the following reasons:

---

### **1. Nonlinear Objective Function**
The objective is to **maximize total revenue**, which is the product of price and demand for both products:

\[
\text{Revenue}_{\text{Product}} = P \times (\text{Intercept} + \text{OwnCoeff} \times P + \text{CrossCoeff} \times P_{\text{Other}})
\]

This leads to a **quadratic objective function**, as the price \( P \) is multiplied by a linear expression of \( P \), resulting in a term like \( P^2 \). Specifically:

\[
\text{Revenue}_{\text{Smartwatch}} = P_{\text{SW}} \times (a + b \times P_{\text{SW}} + c \times P_{\text{EB}})
\]

\[
\text{Revenue}_{\text{Earbuds}} = P_{\text{EB}} \times (d + e \times P_{\text{EB}} + f \times P_{\text{SW}})
\]

When combined, the total revenue includes **quadratic terms** like \( P_{\text{SW}}^2 \), \( P_{\text{EB}}^2 \), and cross-product terms like \( P_{\text{SW}} \times P_{\text{EB}} \). These nonlinear terms prevent the problem from being linear.

---

### **2. Cross-Price Elasticity**
The demand for each product is influenced by the price of the other product through cross-price coefficients. This introduces **interdependence** between the variables \( P_{\text{SW}} \) and \( P_{\text{EB}} \), resulting in a coupled nonlinear system.

---

### **3. No Linear Reformulation is Possible**
A linear reformulation would require transforming all nonlinear terms (e.g., \( P^2 \) and \( P_{\text{SW}} \times P_{\text{EB}} \)) into linear ones. This is not feasible because:

- **Quadratic terms cannot be expressed as linear combinations** without approximations or piecewise linearization, which reduces accuracy and increases complexity.  
- **Cross-product terms \( P_{\text{SW}} \times P_{\text{EB}} \)** further complicate the problem, as they represent genuine nonlinear interactions that cannot be eliminated through simple transformations.

---

### **4. Complexity and Real-World Relevance**
The nonlinearity accurately reflects the real-world relationship between price and demand. Attempting to linearize the problem would:

- Lose critical information about how demand responds to price changes.  
- Produce suboptimal solutions that may significantly underestimate or overestimate revenue.

---

### **Conclusion**
This optimization problem is a nonlinear program due to its **quadratic objective function** and **cross-price interactions**. The problem cannot be linearized without sacrificing accuracy, and solving it as an NLP is essential to capture the true nature of the price-demand relationship.

---

Let me know if you want to illustrate this explanation with examples or visualizations! 😊

# (f) Implement and solve the full model across all 17 weeks using Gurobi. Assume that dynamic pricing is allowed. What is the optimal revenue over the 17-week period?

In [20]:
import pandas as pd
from gurobipy import Model, GRB

# Load the price response data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Create the optimization model
model = Model("Dynamic_Pricing_17_Weeks")

# Add decision variables for each product's price across 17 weeks
P_sw = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Smartwatch")
P_eb = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Earbuds")

# Auxiliary variables for min and max prices
min_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Min_Price_Smartwatch")
max_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Price_Smartwatch")

# Objective function: Maximize total revenue across all 17 weeks
revenue = 0
for week in range(17):
    sw_data = df[(df['Week'] == week + 1) & (df['Product'] == 'TechFit Smartwatch')]
    eb_data = df[(df['Week'] == week + 1) & (df['Product'] == 'PowerSound Earbuds')]

    intercept_sw, own_coeff_sw, cross_coeff_sw = sw_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]
    intercept_eb, own_coeff_eb, cross_coeff_eb = eb_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]

    revenue += (
        P_sw[week] * (intercept_sw + own_coeff_sw * P_sw[week] + cross_coeff_sw * P_eb[week]) +
        P_eb[week] * (intercept_eb + own_coeff_eb * P_eb[week] + cross_coeff_eb * P_sw[week])
    )

model.setObjective(revenue, GRB.MAXIMIZE)

# Add constraints for Table 1

# 1. Weeks 1-4: Static pricing for each product
for week in range(1, 4):
    model.addConstr(P_sw[week] == P_sw[0], f"SW_Static_Weeks1-4_{week}")
    model.addConstr(P_eb[week] == P_eb[0], f"EB_Static_Weeks1-4_{week}")

# 2. Weeks 5-8: Static pricing with $10 reduction compared to Weeks 1-4
for week in range(4, 8):
    model.addConstr(P_sw[week] <= P_sw[0] - 10, f"SW_Reduce_Weeks5-8_{week}")
    model.addConstr(P_eb[week] <= P_eb[0] - 10, f"EB_Reduce_Weeks5-8_{week}")

# 3. Weeks 9-11: Static pricing with $20 increase compared to Weeks 1-4
for week in range(8, 11):
    model.addConstr(P_sw[week] >= P_sw[0] + 20, f"SW_Increase_Weeks9-11_{week}")
    model.addConstr(P_eb[week] >= P_eb[0] + 20, f"EB_Increase_Weeks9-11_{week}")

# 4. Week 12: Lowest price of any week by at least $5 (excluding Week 12 itself)
for week in range(17):
    if week != 11:
        model.addConstr(min_sw <= P_sw[week], f"Min_Price_SW_{week}")
model.addConstr(P_sw[11] <= min_sw - 5, "SW_Lowest_Week12")

# 5. Weeks 13-15: Static pricing between Weeks 1-4 and Weeks 5-8
for week in range(12, 15):
    model.addConstr(P_sw[0] - 10 <= P_sw[week], f"SW_Range_Weeks13-15_Lower_{week}")
    model.addConstr(P_sw[week] <= P_sw[0], f"SW_Range_Weeks13-15_Upper_{week}")

# 6. Week 16: Price at least $4 higher than Week 12 and $6 lower than any other week (excluding Weeks 12 and 16)
model.addConstr(P_sw[15] >= P_sw[11] + 4, "SW_Week16_Higher_Than_Week12")
for week in range(17):
    if week != 11 and week != 15:
        model.addConstr(P_sw[15] <= P_sw[week] - 6, f"SW_Week16_Lower_Than_AnyOtherWeek_{week}")

# 7. Week 17: Highest price of any week by at least $15 (excluding Week 17)
for week in range(16):
    model.addConstr(max_sw >= P_sw[week], f"Max_Price_SW_{week}")
model.addConstr(P_sw[16] >= max_sw + 15, "SW_Highest_Week17")

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    for week in range(17):
        print(f"Week {week + 1} - Optimal Price for TechFit Smartwatch: ${P_sw[week].X:.2f}")
        print(f"Week {week + 1} - Optimal Price for PowerSound Earbuds: ${P_eb[week].X:.2f}")
    print(f"Maximum Total Revenue over 17 Weeks: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 76 rows, 36 columns and 152 nonzeros
Model fingerprint: 0x33fb1596
Model has 51 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  QObjective range [5e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 8 rows and 2 columns
Presolve time: 0.00s
Presolved: 68 rows, 37 columns, 139 nonzeros
Presolved model has 51 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 17
 AA' NZ     : 7.390e+02
 Factor NZ  : 2.175e+03
 Factor Ops : 6.460e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.40154397e+07  1.01313314e

In [21]:
import pandas as pd
from gurobipy import Model, GRB

# Load the price response data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Create the optimization model
model = Model("Dynamic_Pricing_17_Weeks")

# Add decision variables for each product's price across 17 weeks
P_sw = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Smartwatch")
P_eb = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Earbuds")

# Auxiliary variables for min and max prices for both products
min_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Min_Price_Smartwatch")
max_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Price_Smartwatch")
min_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Min_Price_Earbuds")
max_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Price_Earbuds")

# Objective function: Maximize total revenue across all 17 weeks
revenue = 0
for week in range(17):
    sw_data = df[(df['Week'] == week + 1) & (df['Product'] == 'TechFit Smartwatch')]
    eb_data = df[(df['Week'] == week + 1) & (df['Product'] == 'PowerSound Earbuds')]

    intercept_sw, own_coeff_sw, cross_coeff_sw = sw_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]
    intercept_eb, own_coeff_eb, cross_coeff_eb = eb_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]

    revenue += (
        P_sw[week] * (intercept_sw + own_coeff_sw * P_sw[week] + cross_coeff_sw * P_eb[week]) +
        P_eb[week] * (intercept_eb + own_coeff_eb * P_eb[week] + cross_coeff_eb * P_sw[week])
    )

model.setObjective(revenue, GRB.MAXIMIZE)

# Apply constraints for both products

# 1. Weeks 1-4: Static pricing
for week in range(1, 4):
    model.addConstr(P_sw[week] == P_sw[0], f"SW_Static_Weeks1-4_{week}")
    model.addConstr(P_eb[week] == P_eb[0], f"EB_Static_Weeks1-4_{week}")

# 2. Weeks 5-8: Static pricing with $10 reduction
for week in range(4, 8):
    model.addConstr(P_sw[week] <= P_sw[0] - 10, f"SW_Reduce_Weeks5-8_{week}")
    model.addConstr(P_eb[week] <= P_eb[0] - 10, f"EB_Reduce_Weeks5-8_{week}")

# 3. Weeks 9-11: Static pricing with $20 increase
for week in range(8, 11):
    model.addConstr(P_sw[week] >= P_sw[0] + 20, f"SW_Increase_Weeks9-11_{week}")
    model.addConstr(P_eb[week] >= P_eb[0] + 20, f"EB_Increase_Weeks9-11_{week}")

# 4. Week 12: Lowest price of any week by at least $5 (excluding Week 12 itself)
for week in range(17):
    if week != 11:
        model.addConstr(min_sw <= P_sw[week], f"Min_Price_SW_{week}")
        model.addConstr(min_eb <= P_eb[week], f"Min_Price_EB_{week}")
model.addConstr(P_sw[11] <= min_sw - 5, "SW_Lowest_Week12")
model.addConstr(P_eb[11] <= min_eb - 5, "EB_Lowest_Week12")

# 5. Weeks 13-15: Static pricing between Weeks 1-4 and Weeks 5-8
for week in range(12, 15):
    model.addConstr(P_sw[0] - 10 <= P_sw[week], f"SW_Range_Weeks13-15_Lower_{week}")
    model.addConstr(P_sw[week] <= P_sw[0], f"SW_Range_Weeks13-15_Upper_{week}")
    model.addConstr(P_eb[0] - 10 <= P_eb[week], f"EB_Range_Weeks13-15_Lower_{week}")
    model.addConstr(P_eb[week] <= P_eb[0], f"EB_Range_Weeks13-15_Upper_{week}")

# 6. Week 16: Price at least $4 higher than Week 12 and $6 lower than any other week (excluding Weeks 12 and 16)
model.addConstr(P_sw[15] >= P_sw[11] + 4, "SW_Week16_Higher_Than_Week12")
model.addConstr(P_eb[15] >= P_eb[11] + 4, "EB_Week16_Higher_Than_Week12")
for week in range(17):
    if week != 11 and week != 15:
        model.addConstr(P_sw[15] <= P_sw[week] - 6, f"SW_Week16_Lower_Than_AnyOtherWeek_{week}")
        model.addConstr(P_eb[15] <= P_eb[week] - 6, f"EB_Week16_Lower_Than_AnyOtherWeek_{week}")

# 7. Week 17: Highest price of any week by at least $15 (excluding Week 17)
for week in range(16):
    model.addConstr(max_sw >= P_sw[week], f"Max_Price_SW_{week}")
    model.addConstr(max_eb >= P_eb[week], f"Max_Price_EB_{week}")
model.addConstr(P_sw[16] >= max_sw + 15, "SW_Highest_Week17")
model.addConstr(P_eb[16] >= max_eb + 15, "EB_Highest_Week17")

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    for week in range(17):
        print(f"Week {week + 1} - Optimal Price for TechFit Smartwatch: ${P_sw[week].X:.2f}")
        print(f"Week {week + 1} - Optimal Price for PowerSound Earbuds: ${P_eb[week].X:.2f}")
    print(f"Maximum Total Revenue over 17 Weeks: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 132 rows, 38 columns and 264 nonzeros
Model fingerprint: 0xd690a7e1
Model has 51 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  QObjective range [5e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 16 rows and 4 columns
Presolve time: 0.00s
Presolved: 116 rows, 40 columns, 238 nonzeros
Presolved model has 51 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 17
 AA' NZ     : 1.348e+03
 Factor NZ  : 5.470e+03
 Factor Ops : 2.767e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.39431113e+07  1.017166

In [23]:
import pandas as pd
from gurobipy import Model, GRB

# Load the price response data
df = pd.read_csv('/Users/Sam/Downloads/price_response.csv')

# Create the optimization model
model = Model("Dynamic_Pricing_17_Weeks")

# Add decision variables for each product's price across 17 weeks
P_sw = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Smartwatch")
P_eb = model.addVars(17, lb=0, vtype=GRB.CONTINUOUS, name="Price_Earbuds")

# Auxiliary variables for min and max prices for both products
min_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Min_Price_Smartwatch")
max_sw = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Price_Smartwatch")
min_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Min_Price_Earbuds")
max_eb = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Max_Price_Earbuds")

# Objective function: Maximize total revenue across all 17 weeks
revenue = 0
for week in range(17):
    sw_data = df[(df['Week'] == week + 1) & (df['Product'] == 'TechFit Smartwatch')]
    eb_data = df[(df['Week'] == week + 1) & (df['Product'] == 'PowerSound Earbuds')]

    intercept_sw, own_coeff_sw, cross_coeff_sw = sw_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]
    intercept_eb, own_coeff_eb, cross_coeff_eb = eb_data.iloc[0][['Intercept', 'Own_Price_Coefficient', 'Cross_Price_Coefficient']]

    revenue += (
        P_sw[week] * (intercept_sw + own_coeff_sw * P_sw[week] + cross_coeff_sw * P_eb[week]) +
        P_eb[week] * (intercept_eb + own_coeff_eb * P_eb[week] + cross_coeff_eb * P_sw[week])
    )

model.setObjective(revenue, GRB.MAXIMIZE)

# Apply constraints for both products

# 1. Weeks 1-4: Static pricing
for week in range(1, 4):
    model.addConstr(P_sw[week] == P_sw[0], f"SW_Static_Weeks1-4_{week}")
    model.addConstr(P_eb[week] == P_eb[0], f"EB_Static_Weeks1-4_{week}")

# 2. Weeks 5-8: Static pricing with $10 reduction compared to Weeks 1-4
for week in range(5, 8):
    model.addConstr(P_sw[week] == P_sw[4], f"SW_Static_Weeks5-8_{week}")
    model.addConstr(P_eb[week] == P_eb[4], f"EB_Static_Weeks5-8_{week}")
    model.addConstr(P_sw[4] <= P_sw[0] - 10, "SW_Reduce_Week5")
    model.addConstr(P_eb[4] <= P_eb[0] - 10, "EB_Reduce_Week5")

# 3. Weeks 9-11: Static pricing with $20 increase compared to Weeks 1-4
for week in range(9, 11):
    model.addConstr(P_sw[week] == P_sw[8], f"SW_Static_Weeks9-11_{week}")
    model.addConstr(P_eb[week] == P_eb[8], f"EB_Static_Weeks9-11_{week}")
    model.addConstr(P_sw[8] >= P_sw[0] + 20, "SW_Increase_Week9")
    model.addConstr(P_eb[8] >= P_eb[0] + 20, "EB_Increase_Week9")

# 4. Week 12: Lowest price of any week by at least $5 (excluding Week 12 itself)
for week in range(17):
    if week != 11:
        model.addConstr(min_sw <= P_sw[week], f"Min_Price_SW_{week}")
        model.addConstr(min_eb <= P_eb[week], f"Min_Price_EB_{week}")
model.addConstr(P_sw[11] <= min_sw - 5, "SW_Lowest_Week12")
model.addConstr(P_eb[11] <= min_eb - 5, "EB_Lowest_Week12")

# 5. Weeks 13-15: Static pricing between Weeks 1-4 and Weeks 5-8
for week in range(13, 15):
    model.addConstr(P_sw[week] == P_sw[12], f"SW_Static_Weeks13-15_{week}")
    model.addConstr(P_eb[week] == P_eb[12], f"EB_Static_Weeks13-15_{week}")
    model.addConstr(P_sw[0] - 10 <= P_sw[12], "SW_Range_Week13")
    model.addConstr(P_sw[12] <= P_sw[0], "SW_Range_Week13_Upper")
    model.addConstr(P_eb[0] - 10 <= P_eb[12], "EB_Range_Week13")
    model.addConstr(P_eb[12] <= P_eb[0], "EB_Range_Week13_Upper")

# 6. Week 16: Price at least $4 higher than Week 12 and $6 lower than any other week (excluding Weeks 12 and 16)
model.addConstr(P_sw[15] >= P_sw[11] + 4, "SW_Week16_Higher_Than_Week12")
model.addConstr(P_eb[15] >= P_eb[11] + 4, "EB_Week16_Higher_Than_Week12")
for week in range(17):
    if week != 11 and week != 15:
        model.addConstr(P_sw[15] <= P_sw[week] - 6, f"SW_Week16_Lower_Than_AnyOtherWeek_{week}")
        model.addConstr(P_eb[15] <= P_eb[week] - 6, f"EB_Week16_Lower_Than_AnyOtherWeek_{week}")

# 7. Week 17: Highest price of any week by at least $15 (excluding Week 17)
for week in range(16):
    model.addConstr(max_sw >= P_sw[week], f"Max_Price_SW_{week}")
    model.addConstr(max_eb >= P_eb[week], f"Max_Price_EB_{week}")
model.addConstr(P_sw[16] >= max_sw + 15, "SW_Highest_Week17")
model.addConstr(P_eb[16] >= max_eb + 15, "EB_Highest_Week17")

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    for week in range(17):
        print(f"Week {week + 1} - Optimal Price for TechFit Smartwatch: ${P_sw[week].X:.2f}")
        print(f"Week {week + 1} - Optimal Price for PowerSound Earbuds: ${P_eb[week].X:.2f}")
    print(f"Maximum Total Revenue over 17 Weeks: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 138 rows, 38 columns and 276 nonzeros
Model fingerprint: 0xfdcdb8ee
Model has 51 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  QObjective range [5e-01, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 22 rows and 4 columns
Presolve time: 0.00s
Presolved: 116 rows, 36 columns, 234 nonzeros
Presolved model has 51 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 17
 AA' NZ     : 1.246e+03
 Factor NZ  : 4.664e+03
 Factor Ops : 1.981e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -8.39183225e+07  1.010482

# (g) Generate a plot showing the price dynamics for each product over the 17-week period on the same graph. Briefly comment on whether you think GadgetMarket Inc. and its customers would find these price trends favorable. As a customer, is there anything you would find concerning?

# (h) Benchmarking is useful for contextualization. Using Gurobi, what would the optimal revenue be for a dynamic pricing strategy without any price constraints except to ensure prices and demand are non-negative? Alternatively, what would the optimal revenue be if prices were constrained to be the same for all 17 weeks?

# (i) Compare the benefits and drawbacks of (i) the static pricing model, (ii) the unconstrained dynamic pricing model, and (iii) the constrained dynamic pricing model.